${\rm RK}$方程的形式是：
$$p=\frac{RT}{V-b}-\frac{a(T)}{V(V+b)}$$
其中：
$$a(T)=0.42748\frac{R^2T_c^2}{p_c}\alpha(T)$$
$$\alpha(T)=[1+(0.48+1.574\omega-0.176\omega^2)(1-T_r^{0.5})]^2$$
$$b=0.08664\frac{RT_c}{p_c}$$

when p = 453788.0564 Pa
T = 350 K
V = 0.0062105 m^3
Tc = 351.255 K
pc = 57.826 bar
Vc = 122.696 cm^3
w = 0.277

alpha = 1.00323
a = 0.6324966576


In [1]:
import numpy as np
from math import *
import matplotlib.pyplot as plt

class Constants: # 常数类
	R = 8.314

def alpha(omega,T,T_c):
	"""
	alpha(T)函数，传入偏心因子，对比温度
	"""
	T_r = T/T_c
	return (1+(0.48+1.574*omega-0.176*omega*omega)*(1-sqrt(T_r)))**2
def a(omega,T,T_c,p_c):
	"""
	a(T)函数，传入偏心因子，对比温度，临界压强
	"""
	return 0.42748*((Constants.R**2*T_c**2)/(p_c))*alpha(omega,T,T_c)
def b(T_c,p_c):
	"""
	b函数，传入临界温度和压强
	"""
	return 0.08664*Constants.R*T_c/(p_c)

对SRK方程做变形有：
$$b^2+\frac{RT}{p}b+\frac{ab}{pV}=V^2-\frac{RTV}{p}+\frac{a}{p}$$
对于一元二次方程：$ax^2+bx+c=0$：
$$a=1,b=RT/p+a/pV,c=-V^2+RTV/p-a/p$$

In [2]:
def b_real(V,T,p,omega,T_c,p_c)->list:
	"""
	b~real~ —— 计算真实情况下的b值
	"""
	a_ = a(omega,T,T_c,p_c)
	a_s = 1
	b_s = Constants.R*T/p+a_/(p*V)
	c_s = -V*V+Constants.R*T*V/p-a_/p
	b1 = (-b_s+sqrt(b_s**2-4*a_s*c_s))/(2*a_s)
	b2 = (-b_s-sqrt(b_s**2-4*a_s*c_s))/(2*a_s)
	return [a_,b1] # 大于0的值

In [3]:
p = 453788.0564
T = 350.00
V = 0.0062105
Tc = 351.255
pc = 57.826*10**5
Vc = 122.696/(10**6)
w = 0.277

alphat = 1.00323
at = 0.6324966576
print(alpha(w,T,Tc))
print(a(w,T,Tc,pc))
print(b_real(V,T,p,w,Tc,pc))

1.0032300117460562
0.6324966575800304
[0.6324966575800304, 2.095450175313351e-05]


In [4]:
# 文件数据处理
import pandas as pd
def read_excel(filepath,kind):
	excel_file = pd.ExcelFile(filepath)
	active_sheet = excel_file.sheet_names[0]
	df = pd.read_excel(filepath,sheet_name=active_sheet)
	headers = df.columns.tolist()
	if kind == 1: # 列
		columns_data = {}
		for header in headers:
			columns_data[header] = df[header].tolist()
		return columns_data
	elif kind == 0: # 行
		rows_data = []
		for index, row in df.iterrows():
			rows_data.append(row.tolist())
		columns_data = {}
		columns_data[headers[0]] = headers[1:]
		for lists in rows_data:
			columns_data[lists[0]] = lists[1:]
		return columns_data
# 单位 Pa K m^3 (P T V)
PVT_CH2F2 = read_excel('./PVT_data_0_CH2F2.xlsx',1)
PVT_CF3_CH2F = read_excel('./PVT_data_4_CF3_CH2F.xlsx',1)
PVT_CF3_CHF2 = read_excel('./PVT_data_5_CF3_CHF2.xlsx',1)
TIES_OF_MATTER = read_excel('./High precise EOS data 物性.xlsx',0)
# Tc K  pc bar(10^5 Pa) Vc cm^3  w
MATTER_LINK = {}
for i in range(len(TIES_OF_MATTER['matter'])):
	MATTER_LINK[TIES_OF_MATTER['matter'][i]] = i
print(MATTER_LINK)

T_C = np.array(TIES_OF_MATTER['Tc(K)']) # K
V_C = np.array(TIES_OF_MATTER['Vc（cm3)'])/(10**6) # m^3
P_C = np.array(TIES_OF_MATTER['Pc（bar）'])*(10**5) # Pa
M = np.array(TIES_OF_MATTER['M']) # g/mol
OMEGA = np.array(TIES_OF_MATTER['w'])

{'Methane': 0, 'R23 CHF3': 1, 'R32 CH2F2': 2, 'R134a  CF3—CH2F': 3, 'R125  CF3—CHF2': 4}


In [5]:
# 计算a和b
def calcuteAB(PVT:dict, num)->dict:
	keys = list(PVT.keys())
	P,V,T = PVT[keys[0]],PVT[keys[2]],PVT[keys[1]]
	Tc, Vc, pc, omega = T_C[num],V_C[num],P_C[num],OMEGA[num]
	a_list,b_list,b_ideal,Delta_b = [],[],[],[]
	for i in range(len(P)):
		p,v,t = P[i],V[i],T[i]
		temp = b_real(v,t,p,omega,Tc,pc)
		a_list.append(temp[0])
		b_list.append(temp[1])
		b_ideal.append(b(Tc,pc))
		Delta_b.append(temp[1]-b(Tc,pc))
	return {'a':a_list,'b':b_list,'b_ideal':b_ideal,'Delta_b':Delta_b}
def write_column_to_excel(filepath,column_name, data):
	excel_file = pd.ExcelFile(filepath)
	active_sheet = excel_file.sheet_names[0]
	# 读取现有的Excel文件
	df = pd.read_excel(filepath, sheet_name=active_sheet)
	# 添加新列数据
	df[column_name] = data
	df[column_name] = data
	# 将数据写回Excel文件
	with pd.ExcelWriter(filepath, mode='a', if_sheet_exists='replace') as writer:
		df.to_excel(writer, sheet_name=active_sheet, index=False)
AB_CH2F2 = calcuteAB(PVT_CH2F2,2)
AB_CF3_CH2F = calcuteAB(PVT_CF3_CH2F,3)
AB_CF3_CHF2 = calcuteAB(PVT_CF3_CHF2,4)

In [6]:
for i in range(len(AB_CH2F2)):
	write_column_to_excel('./PVT_data_0_CH2F2.xlsx',list(AB_CH2F2.keys())[i],list(AB_CH2F2.values())[i])
for i in range(len(AB_CF3_CHF2)):
	write_column_to_excel('./PVT_data_5_CF3_CHF2.xlsx',list(AB_CF3_CHF2.keys())[i],list(AB_CF3_CHF2.values())[i])
for i in range(len(AB_CF3_CH2F)):
	write_column_to_excel('./PVT_data_4_CF3_CH2F.xlsx',list(AB_CF3_CH2F.keys())[i],list(AB_CF3_CH2F.values())[i])

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset,random_split
class ThreeLayerNN(nn.Module):
	def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
		super(ThreeLayerNN,self).__init__()
		self.fc1 = nn.Linear(input_size, hidden_size1)
		self.relu1 = nn.ReLU()
		self.fc2 = nn.Linear(hidden_size1, hidden_size2)
		self.relu2 = nn.ReLU()
		self.fc3 = nn.Linear(hidden_size2, output_size)

	def forward(self, x):
		out = self.fc1(x)
		out = self.relu1(out)
		out = self.fc2(out)
		out = self.relu2(out)
		out = self.fc3(out)
		return out
# 定义物理损失函数
def physical_loss(output, tr ,pr,vr,M,w): # output 为输出的 b-b_ideal
	return 0

# 输入的数据: Tr,pr,Vr,M,w集合; 输出的集合b-b*的理论值
# Tr 将多个Tr拼接
T_R_SET = []
TRCH2F2 = list(np.array(PVT_CH2F2['Temperature (K)'])/T_C[2])
T_R_SET.append(TRCH2F2)

In [ ]:
import random
import numpy as np
import math
import jittor as jt
from jittor.nn import MSELoss,L1Loss
import matplotlib.pyplot as plt
import pandas as pd
# 使用GPU进行计算
jt.flags.use_cuda = 1

def calculators(omega, Tc, Pc, T):
    """
    定义计算器，用于计算多项式函数的值
    """
    T = np.array(T)  # Make sure T is a NumPy array
    Tc = np.asarray(Tc)
    Pc = np.asarray(Pc)
    # P = RT/(V-b)- a(T)/V(V+b)
    m = 0.480 + 1.574*omega-0.176*(omega**2)
    a = 0.42748*(R**2)*((Tc**2)/Pc)*(1+m*(1-(T/Tc)**0.5))**2
    b = 0.08664*R*Tc/Pc
    dict = {'a':a, 'b':b, 'm':m}
    return dict

def compressibility_factor(P,V,T):
    """
    定义计算器，用于计算压缩因子Z
    """
    P = np.array(P)
    V = np.array(V)
    T = np.array(T)
    Z = P * V / (R * T)
    return Z.tolist()

def calculate_A(T, P, a):
    """
    定义计算器，用于计算A
    """
    T = np.array(T)
    P = np.array(P)
    a = np.array(a)
    return (a * P) / ((R * T) ** 2).tolist()

def calculate_B(A,Z):
    """
    定义计算器，用于计算B
    """
    A = np.array(A)
    Z = np.array(Z) 
    a = Z.tolist()
    b = (Z + A).tolist()
    c = (-(Z**3) + Z**2 - A*Z).tolist()
    roots = []
    for i in range(len(Z)):
        coefficients = [a[i], b[i], c[i]]
        root = np.roots(coefficients)
        positive_root = [r for r in root if r > 0]
        positive_root = positive_root[0] if positive_root else None
        roots.append(positive_root)
    return roots

def read_data(filename, sheet):
    df = pd.read_excel(filename, sheet_name = sheet)
    T = df["Temperature (K)"].tolist()
    P = df['Pressure (kpa)'].tolist()
    V = df['Volume (cm3)'].tolist()
    V = np.array(V)
    V = (V / 1000000).tolist() # 单位转换
    return T, P, V

class DynamicNet(jt.nn.Module):
    def __init__(self): 
        """
        模型初始化，定义5个参数位随机数
        """
        super().__init__()
        self.a = jt.randn(())
        # self.b = jt.randn(())
        # self.c = jt.randn(())
        # self.d = jt.randn(())
        #TODO1：添加一个新的参数e
        # self.e = jt.randn(())

    def execute(self, x):
        """
        模型的前向传播，定义了一个多项式函数，其中包含了5个参数
        y = a + b * x + c * x^2 + d * x^3 + e * x^4 ? + e * x^5 ? (?表示可能存在)
        """
        # todo 拟合曲线表达式
        y = self.a * x
        # y = y + self.b * x 
        # y = y + self.c * x ** 2
        # y = y + self.d * x ** 3
        # for exp in range(4, random.randint(4, 6)):
        #     y = y + self.e * x ** exp
        return y

    def string(self):
        """
        返回多项式模型的字符串表示
        """
        return f'y = {self.a.item()} P/RT'
        # return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'
    
def main():
    omega = 0.277
    global R
    R = 8.314
    Tc = 355
    Pc = 5782600
    b_ls = []
    filename = "D:\EOS\PVT_data1(1).xlsx"
    for sheet_name in ["R32 CH2F2"]:
        T, P, V = read_data(filename, sheet_name)
        ab = calculators(omega, Tc, Pc, T)
        Z = compressibility_factor(P,V,T)
        A = calculate_A(T, P, ab['a'])
        B = calculate_B(A,Z)
        print(B)
        model = DynamicNet()
        #定义损失函数和优化器
        loss_func = MSELoss()
        # loss_func = jt.nn.L1Loss()
        # learning_rate
        learning_rate = 1e-5
        #定义优化器，这里使用了SGD
        optimizer = jt.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        # 准备x和y
        P = np.array(P)
        T = np.array(T)
        x = (P/T).tolist()
        y = B
        for i in range(len(y)):
            if y[i] is None:
                x[i] = 0
                y[i] = 0
        x = [ele for ele in x if ele != 0]
        y = [ele for ele in y if ele != 0]
        for t in range(6000): # 训练6000次
            # 模型的前向传播，计算预测值
            y_pred = model(x)
            # 计算损失
            loss = loss_func(y_pred, y)
            if t % 2000 == 1999:
                print(t, loss.item())
            # jittor的优化器可以直接传入loss，自动计算清空旧的梯度，反向传播得到新的梯度，更新参数
            optimizer.step(loss)
        print(f'Result: {model.string()}')
        b = model.a.item()
        b_ls.append(b)
    print(b_ls)
    plt.plot(b_ls)

if __name__ == '__main__':
    main()